# tfjs-motewan-study



- https://github.com/HCIILAB/SCUT-FBP5500-Database-Release

In [3]:
from google.colab import files
uploaded = files.upload()

Saving SCUT-FBP5500_v2.1.zip to SCUT-FBP5500_v2.1.zip


In [5]:
!unzip SCUT-FBP5500_v2.1.zip

Archive:  SCUT-FBP5500_v2.1.zip
   creating: SCUT-FBP5500_v2/
  inflating: SCUT-FBP5500_v2/README.txt  
 extracting: SCUT-FBP5500_v2/train_test_files.zip  
  inflating: SCUT-FBP5500_v2/All_Ratings.xlsx  
  inflating: SCUT-FBP5500_v2/Images_Sources.xlsx  
   creating: SCUT-FBP5500_v2/facial landmark/
  inflating: SCUT-FBP5500_v2/facial landmark/CF712.pts  
  inflating: SCUT-FBP5500_v2/facial landmark/CF710.pts  
  inflating: SCUT-FBP5500_v2/facial landmark/CF709.pts  
  inflating: SCUT-FBP5500_v2/facial landmark/CF707.pts  
  inflating: SCUT-FBP5500_v2/facial landmark/CF701.pts  
  inflating: SCUT-FBP5500_v2/facial landmark/CF70.pts  
  inflating: SCUT-FBP5500_v2/facial landmark/CF695.pts  
  inflating: SCUT-FBP5500_v2/facial landmark/CF694.pts  
  inflating: SCUT-FBP5500_v2/facial landmark/CF69.pts  
  inflating: SCUT-FBP5500_v2/facial landmark/CF688.pts  
  inflating: SCUT-FBP5500_v2/facial landmark/CF683.pts  
  inflating: SCUT-FBP5500_v2/facial landmark/CF669.pts  
  inflating: SCUT

In [32]:
!ls 

All_Ratings.xlsx  facial landmark  Images_Sources.xlsx	train_test_files
data		  Images	   README.txt		train_test_files.zip


In [0]:
import pandas as pd
ratings=pd.read_table(f"/content/SCUT-FBP5500_v2/train_test_files/All_labels.txt", delimiter=" ", names=["image", "rating"])

In [0]:
ratings["race"] = ratings["image"].apply(lambda x: "Asian" if x[0]=="A" else "Caucasian")
ratings["gender"] = ratings["image"].apply(lambda x: "Male" if x[1]=="M" else "Female")
ratings["target"] = ratings["rating"]/5

In [0]:
ratings_M=ratings[ratings.gender=="Male"]

In [36]:
ratings_M.head()

,image,rating,race,gender,target
1,AM1384.jpg,2.466667,Asian,Male,0.493333
2,AM1234.jpg,2.150000,Asian,Male,0.430000
3,AM1774.jpg,3.750000,Asian,Male,0.750000
6,AM704.jpg,2.483333,Asian,Male,0.496667
7,AM1172.jpg,2.266667,Asian,Male,0.453333


In [37]:
%cd SCUT-FBP5500_v2/

[Errno 2] No such file or directory: 'SCUT-FBP5500_v2/'
/content/SCUT-FBP5500_v2


In [0]:
%mkdir -p data/train/Images
%mkdir -p data/val/Images
%mkdir -p Images/like
%mkdir -p Images/nope
%mkdir -p data/train/Images/like
%mkdir -p data/train/Images/nope
%mkdir -p data/val/Images/like
%mkdir -p data/val/Images/nope

In [0]:
import glob
import numpy as np
import shutil
g = glob.glob('Images/?M*.jpg')
shuf = np.random.permutation(g)
for i in range(2750):
    if i<2000:
        shutil.copy(shuf[i], 'data/train/' + shuf[i])
    else:
        shutil.copy(shuf[i], 'data/val/' + shuf[i])

In [0]:
for row in ratings_M.iterrows():
    if row[1][4] > 0.6:
        shutil.move("Images/"+row[1][0], "Images/like/"+row[1][0])
    else:
        shutil.move("Images/"+row[1][0], "Images/nope/"+row[1][0])

In [0]:
import glob
g = glob.glob('Images/like/?M*.jpg')
shuf = np.random.permutation(g)
for i in range(877):
    if i>877/5:
        shutil.copy(shuf[i], 'data/train/' + shuf[i])
    else:
        shutil.copy(shuf[i], 'data/val/' + shuf[i])

In [0]:
import glob
g = glob.glob('Images/nope/?M*.jpg')
shuf = np.random.permutation(g)
for i in range(1873):
    if i>1873/5:
        shutil.copy(shuf[i], 'data/train/' + shuf[i])
    else:
        shutil.copy(shuf[i], 'data/val/' + shuf[i])

## build model

In [79]:
import keras
mobile_conv = keras.applications.mobilenet.MobileNet(weights='imagenet', include_top=False, input_shape=(160, 160, 3))

17227776/17225924 [==============================] - 1s 0us/step


In [80]:
# Freeze the layers except the last 4 layers
for layer in mobile_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in mobile_conv.layers:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x7f555a405c18> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f555a405a20> False
<keras.layers.convolutional.Conv2D object at 0x7f555a405cf8> False
<keras.layers.normalization.BatchNormalization object at 0x7f555a405588> False
<keras.layers.core.Activation object at 0x7f555a405f28> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f555a405160> False
<keras.layers.convolutional.DepthwiseConv2D object at 0x7f555a4050f0> False
<keras.layers.normalization.BatchNormalization object at 0x7f555a3f1cc0> False
<keras.layers.core.Activation object at 0x7f555a3e8400> False
<keras.layers.convolutional.Conv2D object at 0x7f555a43f0f0> False
<keras.layers.normalization.BatchNormalization object at 0x7f555a43f710> False
<keras.layers.core.Activation object at 0x7f555a419ba8> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f555a479400> False
<keras.layers.convolutional.DepthwiseConv2D object at 0x7f555a4792e8> False
<ke

In [107]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(mobile_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(2, activation='sigmoid'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_160 (Model)   (None, 5, 5, 1024)        3228864   
_________________________________________________________________
flatten_11 (Flatten)         (None, 25600)             0         
_________________________________________________________________
dense_21 (Dense)             (None, 1024)              26215424  
_________________________________________________________________
dropout_11 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 2)                 2050      
Total params: 29,446,338
Trainable params: 27,268,098
Non-trainable params: 2,178,240
_________________________________________________________________


In [108]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
      rescale=1./255)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10

train_dir = "/content/SCUT-FBP5500_v2/data/train/Images"
validation_dir = "/content/SCUT-FBP5500_v2/data/val/Images"
image_size=160

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 2599 images belonging to 2 classes.
Found 1068 images belonging to 2 classes.


In [109]:
# Compile the model
sgd = optimizers.SGD(lr=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('small_last4.h5')

Epoch 1/30
26/25 [==============================] - 22s 855ms/step - loss: 0.9218 - acc: 0.5987 - val_loss: 0.7349 - val_acc: 0.6292
Epoch 2/30
26/25 [==============================] - 13s 491ms/step - loss: 0.7485 - acc: 0.6485 - val_loss: 0.7143 - val_acc: 0.6507
Epoch 3/30
26/25 [==============================] - 13s 492ms/step - loss: 0.6679 - acc: 0.6864 - val_loss: 0.6651 - val_acc: 0.6578
Epoch 4/30
26/25 [==============================] - 13s 495ms/step - loss: 0.6177 - acc: 0.7095 - val_loss: 0.6491 - val_acc: 0.6550
Epoch 5/30
26/25 [==============================] - 13s 494ms/step - loss: 0.5815 - acc: 0.7311 - val_loss: 0.6340 - val_acc: 0.6578
Epoch 6/30
26/25 [==============================] - 13s 491ms/step - loss: 0.5681 - acc: 0.7443 - val_loss: 0.6218 - val_acc: 0.6620
Epoch 7/30
26/25 [==============================] - 13s 486ms/step - loss: 0.5333 - acc: 0.7620 - val_loss: 0.6074 - val_acc: 0.6718
Epoch 8/30
26/25 [==============================] - 13s 492ms/step - 

In [0]:
!pip install -q tensorflowjs

In [112]:
!tensorflowjs_converter --input_format keras small_last4.h5 model

Using TensorFlow backend.


In [113]:
!zip -r model.zip model && ls -l
import google.colab
google.colab.files.download('model.zip')

  adding: model/ (stored 0%)
  adding: model/group1-shard10of29 (deflated 8%)
  adding: model/group1-shard4of29 (deflated 8%)
  adding: model/group1-shard12of29 (deflated 8%)
  adding: model/group1-shard5of29 (deflated 8%)
  adding: model/group1-shard23of29 (deflated 8%)
  adding: model/group1-shard17of29 (deflated 8%)
  adding: model/group1-shard28of29 (deflated 7%)
  adding: model/model.json (deflated 94%)
  adding: model/group1-shard27of29 (deflated 7%)
  adding: model/group1-shard6of29 (deflated 8%)
  adding: model/group1-shard13of29 (deflated 8%)
  adding: model/group1-shard20of29 (deflated 8%)
  adding: model/group1-shard26of29 (deflated 11%)
  adding: model/group1-shard22of29 (deflated 8%)
  adding: model/group1-shard19of29 (deflated 8%)
  adding: model/group1-shard25of29 (deflated 8%)
  adding: model/group1-shard21of29 (deflated 8%)
  adding: model/group1-shard7of29 (deflated 8%)
  adding: model/group1-shard1of29 (deflated 8%)
  adding: model/group1-shard18of29 (deflated 8%)
  